# Monte Carlo simulation for NBA games

In [3]:
#Data comes from URL https://www.basketball-reference.com/teams/MIA/2020_games.html

## Imports

In [12]:
import pandas as pd
import numpy as np
import random as rd

## Uploading data to dataframe and naming based on team

In [5]:
MH = pd.read_csv('Miami20_stats.csv')
MH.name = 'Miami'
BC = pd.read_csv('Boston20_stats.csv')
BC.name = 'Boston'

## Function to scale Pts scored and Pts given up for the specific matchup
Since sports are matchup based gives a higher weight for the specific matchup when making the gaussian

In [6]:
def adj_Value(df, name):
    for i in range(len(df)):
        var = df['Opponent'][i] == name
    
        if var == True:
            df['Tm'][i] = df['Tm'][i]*2
            df['Opp'][i] = df['Opp'][i]*2
    return df.sort_values(by = 'Tm')

In [7]:
#Finds index of opposing team to apply the value adjusting for matchup
def matchup_adj(T1,T2):
    T1_name = list(set(T2['Opponent']).difference(T1['Opponent']))
    T2_name = list(set(T1['Opponent']).difference(T2['Opponent']))
    
    T1_adj = adj_Value(T1, T2_name)
    T2_adj = adj_Value(T2, T1_name)
    
    return(T1_adj, T2_adj)

## Monte Carlo Simulation
Simulates each 'game' and returns 1,-1,0 based on Win, Loss, Tie

A guassian is made using the pts scored by team one and the pts given by the other team

In [8]:
def GameSim(T1,T2):
    T1, T2 = matchup_adj(T1,T2)
    
    T1_adjusted = T1.drop(T1.columns[[0,1,2,3,4,5,6,7,10,11,12]], axis=1)
    T1_mean = np.mean(T1_adjusted)
    T1_std = np.mean(T1_adjusted)
    
    T2_adjusted = T2.drop(T2.columns[[0,1,2,3,4,5,6,7,10,11,12]], axis=1)
    T2_mean = np.mean(T2_adjusted)
    T2_std = np.mean(T2_adjusted)
    
    T1_gauss = (rd.gauss(T1_mean[0],T1_std[0]) + rd.gauss(T2_mean[1],T2_std[1]))/2
    T2_gauss = (rd.gauss(T2_mean[0],T2_std[0]) + rd.gauss(T1_mean[1],T1_std[1]))/2
    
    if int(round(T1_gauss)) > int(round(T2_gauss)):
        return 1
    elif int(round(T1_gauss)) < int(round(T2_gauss)):
        return -1
    else:
        return 0

Runs the simulation and documents results from random sampling gaussian for N number of games between Teams 1 and 2

In [10]:
def Simulation(T1,T2,N):
    results = []
    T1_win = 0
    T2_win = 0
    tie = 0
    
    for i in range(N):
        g = GameSim(T1,T2)
        results.append(g)
        
        if g == 1: 
            T1_win +=1
        elif g == -1:
            T2_win +=1
        else: 
            tie +=1
    
    print(T1.name + ' Win', T1_win/(T1_win+T2_win+tie),'%')
    print(T2.name + ' Win', T2_win/(T1_win+T2_win+tie),'%')
    print('Tie', tie/(T1_win+T2_win+tie),'%')

In [16]:
Simulation(MH,BC,10000)

Miami Win 0.4995 %
Boston Win 0.4961 %
Tie 0.0044 %
